<a href="https://colab.research.google.com/github/rahiakela/advanced-natural-language-processing-with-tensorflow-2/blob/main/1-essentials-of-nlp/1_text_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text Normalization

To understand how to process text, it is important to understand the general
workflow for NLP.

<img src='https://github.com/rahiakela/img-repo/blob/master/advanced-nlp-with-tensorflow-2/text-processing-workflow.png?raw=1' width='800'/>

The first two steps of the process in the preceding diagram involve collecting labeled data. A supervised model or even a semi-supervised model needs data to operate.

The next step is usually normalizing and featurizing the data. Models have a hard time processing text data as is. There is a lot of hidden structure in a given text that needs to be processed and exposed. These two steps focus on that. 

The last step is building a model with the processed inputs. While NLP has some unique models, this chapter will use only a simple deep neural network and focus more on the normalization and vectorization/featurization. Often, the last three stages operate in a cycle, even though the diagram may give the impression of linearity.

## Setup

In [ ]:
%%shell

pip install stopwordsiso
# pip install stanfordnlp
pip install stanza  # StanfordNLP has become https://github.com/stanfordnlp/stanza/

In [ ]:
%tensorflow_version 2.x     # magic command instructing to use TensorFlow version 2+
import tensorflow as tf
#from tf.keras.models import Sequential
#from tf.keras.layers import Dense
import os
import io
import re

import pandas as pd 
import stopwordsiso as stopwords
import stanza
import stanfordnlp as snlp
# en = snlp.download('en')
en = stanza.download('en')

tf.__version__

# Data collection

**The first step of any Machine Learning (ML) project is to obtain a dataset.**

We will be using the SMS Spam Collection dataset made available by University of California, Irvine.

In [ ]:
# Download the zip file
path_to_zip = tf.keras.utils.get_file("smsspamcollection.zip",
                  origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip",
                  extract=True)

# Unzip the file into a folder
!unzip $path_to_zip -d data

In [ ]:
# optional step - helps if colab gets disconnected
# from google.colab import drive
# drive.mount('/content/drive')

Reading the data file is trivial.

In [ ]:
# Let's see if we read the data correctly
# lines = io.open('/content/drive/My Drive/colab-data/SMSSpamCollection').read().strip().split('\n')
lines = io.open('/content/data/SMSSpamCollection').read().strip().split('\n')
lines[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
lines[2]

"spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

## Pre-process Data

The next step is to split each line into two columns – one with the text of the message and the other as the label. While we are separating these labels, we will also convert the labels to numeric values. Since we are interested in predicting spam messages, we can assign a value of 1 to the spam
messages. A value of 0 will be assigned to legitimate messages.

In [ ]:
spam_dataset = []
spam_count = 0
ham_count = 0
for line in lines:
  label, text = line.split('\t')
  if label.lower().strip() == 'spam':
    spam_dataset.append((1, text.strip()))
    spam_count += 1
  else:
    spam_dataset.append(((0, text.strip())))
    ham_count += 1

spam_dataset[:5]

[(0,
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'),
 (0, 'Ok lar... Joking wif u oni...'),
 (1,
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"),
 (0, 'U dun say so early hor... U c already then say...'),
 (0, "Nah I don't think he goes to usf, he lives around here though")]

In [ ]:
print("Spam: ", spam_count, ", Ham: ", ham_count)

Spam:  747 , Ham:  4827


Now the dataset is ready for further processing in the pipeline.

# Data Normalization

Text normalization is a pre-processing step aimed at improving the quality
of the text and making it suitable for machines to process. 

Four main steps in text normalization are:

- case normalization, 
- tokenization and stop word removal,
- Parts-of-Speech (POS) tagging, 
- and stemming.

## Case normalization

**Case normalization applies to languages that use uppercase and lowercase letters.**

All languages based on the Latin alphabet or the Cyrillic alphabet (Russian,
Mongolian, and so on) use upper- and lowercase letters. Other languages
that sometimes use this are Greek, Armenian, Cherokee, and Coptic. 

In case normalization, all letters are converted to the same case. It is quite helpful in semantic use cases. However, in other cases, this may hinder performance.

### Preprocessing case normalized data

Let's build a baseline model with three simple features:

- Number of characters in the message
- Number of capital letters in the message
- Number of punctuation symbols in the message

In [ ]:
# To do so, first, we will convert the data into a pandas DataFrame
df = pd.DataFrame(spam_dataset, columns=['Spam', 'Message'])

In [ ]:
df.head()

,Spam,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Next, let's build some simple functions that can count the length of the message, and the numbers of capital letters and punctuation symbols. Python's regular expression package, re, will be used to implement these:

In [ ]:
# Normalization functions

def message_length(x):
  # returns total number of characters
  return len(x)

def num_capitals(x):
  # get count of capital letters
  _, count = re.subn(r'[A-Z]', '', x) # only works in english
  return count

def num_punctuation(x):
  # get count the number of punctuation symbols
  _, count = re.subn(r'\W', '', x)
  return count

Additional feature columns will be added to the DataFrame, and then the set will
be split into test and train sets:

In [ ]:
df['Capitals'] = df['Message'].apply(num_capitals)
df['Punctuation'] = df['Message'].apply(num_punctuation)
df['Length'] = df['Message'].apply(message_length)

In [ ]:
df.head()

,Spam,Message,Capitals,Punctuation,Length
0,0,"Go until jurong point, crazy.. Available only ...",3,28,111
1,0,Ok lar... Joking wif u oni...,2,11,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,10,33,155
3,0,U dun say so early hor... U c already then say...,2,16,49
4,0,"Nah I don't think he goes to usf, he lives aro...",2,14,61


In [ ]:
df.describe()

,Spam,Capitals,Punctuation,Length
count,5574.000000,5574.000000,5574.000000,5574.000000
mean,0.134015,5.621636,18.942591,80.443488
std,0.340699,11.683233,14.825994,59.841746
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,36.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,27.000000,122.000000
max,1.000000,129.000000,253.000000,910.000000


Now let's split the dataset into training and test sets, with
80% of the records in the training set and the rest in the test set.

In [ ]:
train=df.sample(frac=0.8,random_state=42) #random state is a seed value
test=df.drop(train.index)

In [ ]:
train.describe()

,Spam,Capitals,Punctuation,Length
count,4459.000000,4459.000000,4459.000000,4459.000000
mean,0.132765,5.519399,18.886522,80.316439
std,0.339359,11.405424,14.602023,59.346407
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,35.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,27.000000,122.000000
max,1.000000,129.000000,253.000000,910.000000


In [ ]:
test.describe()

,Spam,Capitals,Punctuation,Length
count,1115.000000,1115.000000,1115.000000,1115.000000
mean,0.139013,6.030493,19.166816,80.951570
std,0.346116,12.731059,15.694599,61.807655
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,36.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,28.000000,123.000000
max,1.000000,127.000000,195.000000,790.000000


Further more, labels will be removed from both the training and test sets:

In [ ]:
x_train = train[['Length', 'Punctuation', 'Capitals']]
y_train = train[['Spam']]

x_test = test[['Length', 'Punctuation', 'Capitals']]
y_test = test[['Spam']]

In [ ]:
x_train.describe()

,Length,Punctuation,Capitals
count,4459.000000,4459.000000,4459.000000
mean,80.316439,18.886522,5.519399
std,59.346407,14.602023,11.405424
min,2.000000,0.000000,0.000000
25%,35.000000,8.000000,1.000000
50%,61.000000,15.000000,2.000000
75%,122.000000,27.000000,4.000000
max,910.000000,253.000000,129.000000


In [ ]:
x_test.describe()

,Length,Punctuation,Capitals
count,1115.000000,1115.000000,1115.000000
mean,80.951570,19.166816,6.030493
std,61.807655,15.694599,12.731059
min,2.000000,0.000000,0.000000
25%,36.000000,8.000000,1.000000
50%,61.000000,15.000000,2.000000
75%,123.000000,28.000000,4.000000
max,790.000000,195.000000,127.000000


### Modeling case normalized data

We will use a very simple model, as the objective is to show different basic NLP data processing techniques more than modeling. Here, we want to see if three simple features can aid in the classification of spam. As more features are added, passing them through the same model will help in seeing if the
featurization aids or hampers the accuracy of the classification.

In [ ]:
# Basic 1-layer neural network model for evaluation
def make_model(input_dims=3, num_units=12):
  model = tf.keras.Sequential()

  # Adds a densely-connected layer with 12 units to the model:
  model.add(tf.keras.layers.Dense(num_units, 
                                  input_dim=input_dims, 
                                  activation='relu'))

  # Add a sigmoid layer with a binary output unit:
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model

This model uses binary cross-entropy for computing loss and the Adam optimizer
for training. The key metric, given that this is a binary classification problem, is accuracy.

We can train our simple baseline model with only three features like so:

In [ ]:
model = make_model()

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 3s 2ms/step - loss: 1.0062 - accuracy: 0.8804
Epoch 2/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2998 - accuracy: 0.9176
Epoch 3/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2208 - accuracy: 0.9406
Epoch 4/10
446/446 [==============================] - 1s 2ms/step - loss: 0.1931 - accuracy: 0.9412
Epoch 5/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2071 - accuracy: 0.9346
Epoch 6/10
446/446 [==============================] - 1s 2ms/step - loss: 0.1818 - accuracy: 0.9388
Epoch 7/10
446/446 [==============================] - 1s 2ms/step - loss: 0.1990 - accuracy: 0.9327
Epoch 8/10
446/446 [==============================] - 1s 2ms/step - loss: 0.1799 - accuracy: 0.9435
Epoch 9/10
446/446 [==============================] - 1s 2ms/step - loss: 0.1778 - accuracy: 0.9442
Epoch 10/10
446/446 [==============================] - 1s 2ms/step - loss: 0.1793 - accuracy: 0.9394

This is not bad as our three simple features help us get to 93% accuracy. A quick check shows that there are 592 spam messages in the test set, out of a total of 4,459. So, this model is doing better than a very simple model that guesses everything as not spam.

That model would have an accuracy of 87%. This number may be
surprising but is fairly common in classification problems where there is a severe class imbalance in the data. Evaluating it on the training set gives an accuracy of around 93.27%:

In [ ]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 1ms/step - loss: 0.1936 - accuracy: 0.9417


[0.19357681274414062, 0.9417040348052979]

Please note that the actual performance you see may be slightly different due to the data splits and computational vagaries. 

A quick verification can be performed by plotting the confusion matrix to see the performance:

In [ ]:
y_train_pred = model.predict_classes(x_train)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
# confusion matrix
tf.math.confusion_matrix(tf.constant(y_train.Spam), y_train_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3756,  111],
       [ 157,  435]], dtype=int32)>

This shows that 3,666 out of 3,867 regular messages were classified correctly, while 353 out of 592 spam messages were classified correctly. Again, you may get a slightly different result.

|  | **Predicted Not Spam** | **Predicted Spam** |
| --- | --- | --- |
| **Actual Not Spam** | 3777 | 90 |
| **Actual Spam** | 186 | 406 |

We can get calculation as follow:

|  | **Predicted Not Spam** | **Predicted Spam** | |
| --- | --- | --- | |
| **Actual Not Spam** | 3777 | 90 | 3777 + 90 = 3867 |
| **Actual Spam**     | 186  | 406 |  186 + 406 = 592  |

So confusion matrix show us that if we reduce the value 201 and 239 then accuracy would be increased.

In [ ]:
sum(y_train_pred)

array([546], dtype=int32)

In [ ]:
y_test_pred = model.predict_classes(x_test)
tf.math.confusion_matrix(tf.constant(y_test.Spam), y_test_pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[939,  21],
       [ 44, 111]], dtype=int32)>

### Excersize

To test the value of the features, try re-running the model by removing one of the features, such as punctuation or a number of capital letters, to get a sense of their contribution to the model.

In [ ]:
x_train = train[['Length', 'Punctuation']]
y_train = train[['Spam']]

x_test = test[['Length', 'Punctuation']]
y_test = test[['Spam']]

x_train.describe()

,Length,Punctuation
count,4459.000000,4459.000000
mean,80.316439,18.886522
std,59.346407,14.602023
min,2.000000,0.000000
25%,35.000000,8.000000
50%,61.000000,15.000000
75%,122.000000,27.000000
max,910.000000,253.000000


In [ ]:
x_test.describe()

,Length,Punctuation
count,1115.000000,1115.000000
mean,80.951570,19.166816
std,61.807655,15.694599
min,2.000000,0.000000
25%,36.000000,8.000000
50%,61.000000,15.000000
75%,123.000000,28.000000
max,790.000000,195.000000


In [ ]:
model1 = make_model(input_dims=2)
model1.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 1s 2ms/step - loss: 7.1731 - accuracy: 0.3518
Epoch 2/10
446/446 [==============================] - 1s 2ms/step - loss: 0.4533 - accuracy: 0.8673
Epoch 3/10
446/446 [==============================] - 1s 2ms/step - loss: 0.4089 - accuracy: 0.8768
Epoch 4/10
446/446 [==============================] - 1s 2ms/step - loss: 0.3434 - accuracy: 0.8893
Epoch 5/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2983 - accuracy: 0.8974
Epoch 6/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2918 - accuracy: 0.8949
Epoch 7/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2838 - accuracy: 0.8901
Epoch 8/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2665 - accuracy: 0.8904
Epoch 9/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2549 - accuracy: 0.8967
Epoch 10/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2503 - accuracy: 0.8886

In [ ]:
model1.evaluate(x_test, y_test)

35/35 [==============================] - 0s 2ms/step - loss: 0.2516 - accuracy: 0.8924


[0.25163596868515015, 0.8923766613006592]

In [ ]:
y_train_pred = model1.predict_classes(x_train)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
# confusion matrix
tf.math.confusion_matrix(tf.constant(y_train.Spam), y_train_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3646,  221],
       [ 248,  344]], dtype=int32)>

Now trying to remove Punctuation letters.

In [ ]:
x_train = train[['Length', 'Capitals']]
y_train = train[['Spam']]

x_test = test[['Length', 'Capitals']]
y_test = test[['Spam']]

In [ ]:
model2 = make_model(input_dims=2)
model2.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 1s 2ms/step - loss: 6.8152 - accuracy: 0.8690
Epoch 2/10
446/446 [==============================] - 1s 2ms/step - loss: 0.4750 - accuracy: 0.8870
Epoch 3/10
446/446 [==============================] - 1s 2ms/step - loss: 0.3839 - accuracy: 0.9137
Epoch 4/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2838 - accuracy: 0.9249
Epoch 5/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2678 - accuracy: 0.9188
Epoch 6/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2663 - accuracy: 0.9143
Epoch 7/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2648 - accuracy: 0.9135
Epoch 8/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2385 - accuracy: 0.9181
Epoch 9/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2361 - accuracy: 0.9216
Epoch 10/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2296 - accuracy: 0.9196

In [ ]:
model2.evaluate(x_test, y_test)

35/35 [==============================] - 0s 1ms/step - loss: 0.2676 - accuracy: 0.9121


[0.2676241099834442, 0.9121076464653015]

In [ ]:
y_train_pred = model2.predict_classes(x_train)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
# confusion matrix
tf.math.confusion_matrix(tf.constant(y_train.Spam), y_train_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3724,  143],
       [ 215,  377]], dtype=int32)>

We observe that removing one of the features punctuation letters, It contribute to the model accuracy by increasing upto 90%.

## Tokenization normalization

This step takes a piece of text and converts it into a list of tokens. If the input is a sentence, then separating the words would be an example of tokenization. Depending on the model, different granularities can be chosen. At the lowest level, each character could become a token. In some cases, entire sentences of paragraphs can be considered as a token:

<img src='https://github.com/rahiakela/img-repo/blob/master/advanced-nlp-with-tensorflow-2/sentence-tokenizing.png?raw=1' width='800'/>

The preceding diagram shows two ways a sentence can be tokenized. One way to
tokenize is to chop a sentence into words. Another way is to chop into individual characters. However, this can be a complex proposition in some languages such as Japanese and Mandarin.


Many languages use a word separator, a space, to separate words. This makes the
task of tokenizing on words trivial. However, there are other languages that do not use any markers or separators between words. Some examples of such languages are Japanese and Chinese. In such languages, the task is referred to as segmentation.

Fortunately, most languages are not as complex as Japanese and use spaces to
separate words. In Python, splitting by spaces is trivial.

In [ ]:
sentence = 'Go until jurong point, crazy.. Available only in bugis n great world'
sentence.split()

['Go',
 'until',
 'jurong',
 'point,',
 'crazy..',
 'Available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world']

The two lines(`point,` and `crazy..`) in the preceding output show that the naïve approach in Python will result in punctuation being included in the words, among other issues. Consequently, this step is done through a library like StanfordNLP.

This package provides capabilities for tokenization, POS tagging, and lemmatization out of the box. To start with tokenization, we instantiate a pipeline and tokenize a sample text to see how this works:

In [ ]:
en = snlp.Pipeline(lang="en", processors="tokenize")

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


For now, only tokenization of text is desired, so only the tokenizer is used:

In [ ]:
tokenized = en(sentence)
len(tokenized.sentences)

2

This shows that the tokenizer correctly divided the text into two sentences.

To investigate what words were removed, the following code can be used:

In [ ]:
for snt in tokenized.sentences:
  for word in snt.tokens:
    print(word.text)
  print("<End of Sentence>")

Go
until
jurong
point
,
crazy
..
<End of Sentence>
Available
only
in
bugis
n
great
world
<End of Sentence>


Punctuation marks were separated out into their own words. Text was split into multiple sentences. This is an improvement over only using spaces to split. In some applications, removal of punctuation may be required.

### Japanese Tokenization Example

Consider the preceding example of Japanese. To see the performance of StanfordNLP on Japanese tokenization, the following piece of code can be used:

In [ ]:
jp = snlp.download('ja')

Using the default treebank "ja_gsd" for language "ja".
Would you like to download the models for: ja_gsd now? (Y/n)
Y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/ja_gsd_models.zip


100%|██████████| 219M/219M [00:22<00:00, 9.61MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/ja_gsd_models.zip
Extracting models file for: ja_gsd
Cleaning up...Done.


Next, a Japanese pipeline will be instantiated and the words will be processed:

In [ ]:
jp = snlp.Pipeline(lang="ja", processors="tokenize")
jp_line = jp("選挙管理委員会")

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/ja_gsd_models/ja_gsd_tokenizer.pt', 'lang': 'ja', 'shorthand': 'ja_gsd', 'mode': 'predict'}
Done loading processors!
---


You may recall that the Japanese text reads Election Administration Committee.
Correct tokenization should produce three words, where first two should be two
characters each, and the last word is three characters:

In [ ]:
for snt in jp_line.sentences:
  for word in snt.tokens:
    print(word.text)

選挙
管理
委員会


This matches the expected output. StanfordNLP supports 53 languages, so the same
code can be used for tokenizing any language that is supported.

Coming back to the spam detection example, a new feature can be implemented that
counts the number of words in the message using this tokenization functionality.

### Modeling tokenized data

It is possible that spam messages have different numbers of words than regular
messages. The first step is to define a method to compute the number of words:

In [ ]:
def word_counts(x, pipeline=en):
  doc = pipeline(x)
  count = sum( [ len(sentence.tokens) for sentence in doc.sentences] )
  return count

In [ ]:
en = snlp.Pipeline(lang='en', processors='tokenize')
df['Words'] = df['Message'].apply(word_counts)

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


In [ ]:
df.head()

,Spam,Message,Capitals,Punctuation,Length,Words
0,0,"Go until jurong point, crazy.. Available only ...",3,28,111,24
1,0,Ok lar... Joking wif u oni...,2,11,29,8
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,10,33,155,34
3,0,U dun say so early hor... U c already then say...,2,16,49,13
4,0,"Nah I don't think he goes to usf, he lives aro...",2,14,61,15


Next, using the train and test splits, add a column for the word count feature:

In [ ]:
#train=df.sample(frac=0.8,random_state=42) #random state is a seed value
#test=df.drop(train.index)

train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)

In [ ]:
x_train = train[['Length', 'Punctuation', 'Capitals', 'Words']]
y_train = train[['Spam']]

x_test = test[['Length', 'Punctuation', 'Capitals' , 'Words']]
y_test = test[['Spam']]

In [ ]:
model = make_model(input_dims=4)
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 1s 2ms/step - loss: 3.7059 - accuracy: 0.6091
Epoch 2/10
446/446 [==============================] - 1s 2ms/step - loss: 0.4739 - accuracy: 0.8881
Epoch 3/10
446/446 [==============================] - 1s 2ms/step - loss: 0.3698 - accuracy: 0.8862
Epoch 4/10
446/446 [==============================] - 1s 2ms/step - loss: 0.3376 - accuracy: 0.8912
Epoch 5/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2988 - accuracy: 0.8983
Epoch 6/10
446/446 [==============================] - 1s 2ms/step - loss: 0.3108 - accuracy: 0.8893
Epoch 7/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2763 - accuracy: 0.8966
Epoch 8/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2697 - accuracy: 0.9017
Epoch 9/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2698 - accuracy: 0.9004
Epoch 10/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2689 - accuracy: 0.8981

There is only a marginal improvement in accuracy. One hypothesis is that the
number of words is not useful. It would be useful if the average number of words in spam messages were smaller or larger than regular messages.

In [ ]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 1ms/step - loss: 0.2667 - accuracy: 0.8870


[0.2666502594947815, 0.8869954943656921]

Using pandas, this can be quickly verified:

In [ ]:
train.loc[train.Spam == 1].describe()

,Spam,Capitals,Punctuation,Length,Words
count,592.0,592.000000,592.000000,592.000000,592.000000
mean,1.0,15.320946,29.086149,138.856419,29.511824
std,0.0,11.635105,7.083572,28.079980,7.474256
min,1.0,0.000000,2.000000,13.000000,3.000000
25%,1.0,7.000000,26.000000,132.000000,26.000000
50%,1.0,14.000000,30.000000,149.000000,30.000000
75%,1.0,21.000000,34.000000,157.000000,35.000000
max,1.0,128.000000,49.000000,197.000000,49.000000


Let's compare the preceding results to the statistics for regular messages:

In [ ]:
train.loc[train.Spam == 0].describe()

,Spam,Capitals,Punctuation,Length,Words
count,3867.0,3867.000000,3867.000000,3867.000000,3867.000000
mean,0.0,4.018878,17.325058,71.354538,17.344194
std,0.0,10.599291,14.826644,57.755351,13.811278
min,0.0,0.000000,0.000000,2.000000,1.000000
25%,0.0,1.000000,8.000000,33.000000,8.000000
50%,0.0,2.000000,13.000000,53.000000,13.000000
75%,0.0,3.000000,23.000000,92.000000,22.000000
max,0.0,129.000000,253.000000,910.000000,209.000000


Some interesting patterns can quickly be seen. Spam messages usually have much
less deviation from the mean. Focus on the Capitals feature column. It shows that regular messages use far fewer capitals than spam messages.

This quick check yields an indication as to why adding the word features wasn't that useful. However, there are a couple of things to consider still. 

First, the tokenization model split out punctuation marks as words. Ideally, these words should be removed from the word counts as the punctuation feature is showing that spam messages use a lot more punctuation characters.

Secondly, languages have some common words that are usually excluded. This is called stop word removal.

## Stop Word Removal normalization

Stop word removal involves removing common words such as articles (the, an)
and conjunctions (and, but), among others. In the context of information retrieval or search, these words would not be helpful in identifying documents or web pages that would match the query.

The step after text normalization is vectorization.The key step in vectorization is to build a vocabulary or dictionary of all the tokens. The size of this vocabulary can be reduced by removing stop words. While training and evaluating models, removing stop words reduces the number of computation steps that need to be performed. Hence, the removal of stop words can yield benefits in terms of computation speed and storage space.

Many NLP packages provide lists of stop words. These can be removed from the
text after tokenization. For this example, we will use
an open source package called stopwordsiso.

English language stop words can be checked as well to get an idea of some of the
words.

In [ ]:
import stopwordsiso as stopwords

stopwords.langs()

{'af',
 'ar',
 'bg',
 'bn',
 'br',
 'ca',
 'cs',
 'da',
 'de',
 'el',
 'en',
 'eo',
 'es',
 'et',
 'eu',
 'fa',
 'fi',
 'fr',
 'ga',
 'gl',
 'gu',
 'ha',
 'he',
 'hi',
 'hr',
 'hu',
 'hy',
 'id',
 'it',
 'ja',
 'ko',
 'ku',
 'la',
 'lt',
 'lv',
 'mr',
 'ms',
 'nl',
 'no',
 'pl',
 'pt',
 'ro',
 'ru',
 'sk',
 'sl',
 'so',
 'st',
 'sv',
 'sw',
 'th',
 'tl',
 'tr',
 'uk',
 'ur',
 'vi',
 'yo',
 'zh',
 'zu'}

In [ ]:
sorted(stopwords.stopwords('en'))[:50]

["'ll",
 "'tis",
 "'twas",
 "'ve",
 '10',
 '39',
 'a',
 "a's",
 'able',
 'ableabout',
 'about',
 'above',
 'abroad',
 'abst',
 'accordance',
 'according',
 'accordingly',
 'across',
 'act',
 'actually',
 'ad',
 'added',
 'adj',
 'adopted',
 'ae',
 'af',
 'affected',
 'affecting',
 'affects',
 'after',
 'afterwards',
 'ag',
 'again',
 'against',
 'ago',
 'ah',
 'ahead',
 'ai',
 "ain't",
 'aint',
 'al',
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'alongside',
 'already',
 'also']

Given that tokenization was already implemented in the preceding word_counts()
method, the implementation of that method can be updated to include removing
stop words. However, all the stop words are in lowercase.

In this case, tokens need to be converted to lowercase to effectively remove them:

In [ ]:
en_sw = stopwords.stopwords('en')

def word_counts(x, pipeline=en):
  doc = pipeline(x)
  count = 0
  for sentence in doc.sentences:
    for token in sentence.tokens:
        if token.text.lower() not in en_sw:
          count += 1
  return count

In [ ]:
en = snlp.Pipeline(lang='en', processors='tokenize')
df['Words'] = df['Message'].apply(word_counts)

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


In [ ]:
train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)

In [ ]:
train.describe()

,Spam,Capitals,Punctuation,Length,Words
count,4459.000000,4459.000000,4459.000000,4459.000000,4459.000000
mean,0.132765,5.519399,18.886522,80.316439,9.312178
std,0.339359,11.405424,14.602023,59.346407,8.019288
min,0.000000,0.000000,0.000000,2.000000,0.000000
25%,0.000000,1.000000,8.000000,35.000000,4.000000
50%,0.000000,2.000000,15.000000,61.000000,7.000000
75%,0.000000,4.000000,27.000000,122.000000,13.000000
max,1.000000,129.000000,253.000000,910.000000,147.000000


The maximum number of words has also reduced from 209 to 147. The standard deviation of regular messages is about the same as its mean, indicating that there is a lot of variation in the number of words in regular messages.

In [ ]:
train.head()

,Spam,Message,Capitals,Punctuation,Length,Words
3690,0,You still coming tonight?,1,4,25,3
3527,0,"""HEY BABE! FAR 2 SPUN-OUT 2 SPK AT DA MO... DE...",107,48,161,36
724,0,Ya even those cookies have jelly on them,1,7,40,3
3370,0,Sorry i've not gone to that place. I.ll do so ...,3,17,69,5
468,0,When are you going to ride your bike?,1,8,37,3


### Modeling data with stop words removed

Now, let's see if this helps us train a model and improve its accuracy.

Now that the feature without stop words is computed, it can be added to the model to see its impact:

In [ ]:
x_train = train[['Length', 'Punctuation', 'Capitals', 'Words']]
y_train = train[['Spam']]

x_test = test[['Length', 'Punctuation', 'Capitals' , 'Words']]
y_test = test[['Spam']]

model = make_model(input_dims=4)
#model = make_model(input_dims=3)

model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 1s 2ms/step - loss: 3.4133 - accuracy: 0.8159
Epoch 2/10
446/446 [==============================] - 1s 2ms/step - loss: 0.4160 - accuracy: 0.8778
Epoch 3/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2975 - accuracy: 0.8967
Epoch 4/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2404 - accuracy: 0.9202
Epoch 5/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2389 - accuracy: 0.9157
Epoch 6/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2401 - accuracy: 0.9141
Epoch 7/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2294 - accuracy: 0.9238
Epoch 8/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2059 - accuracy: 0.9359
Epoch 9/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2290 - accuracy: 0.9195
Epoch 10/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2090 - accuracy: 0.9305

This accuracy reflects a slight improvement over the previous model:

In [ ]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 1ms/step - loss: 0.2311 - accuracy: 0.9247


[0.23106859624385834, 0.9246636629104614]

## Part-of-speech tagging(POS) normalization

Languages have a grammatical structure. In most languages, words can be
categorized primarily into verbs, adverbs, nouns, and adjectives. The objective of this part of the processing step is to take a piece of text and tag each word token with a POS identifier.

Note that this makes sense only in the case of word-level
tokens. Commonly, the Penn Treebank POS tagger is used by libraries including
StanfordNLP to tag words. By convention, POS tags are added by using a code
after the word, separated by a slash. As an example, NNS is the tag for a plural noun.If the words goats was encountered, it would be represented as goats/NNS.

In [ ]:
en = stanza.Pipeline(lang="en")

txt = "Yo you around? A friend of mine's lookin."
pos = en(txt)

2021-03-04 09:03:48 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-03-04 09:03:48 INFO: Use device: gpu
2021-03-04 09:03:48 INFO: Loading: tokenize
2021-03-04 09:03:56 INFO: Loading: pos
2021-03-04 09:03:56 INFO: Loading: lemma
2021-03-04 09:03:56 INFO: Loading: depparse
2021-03-04 09:03:57 INFO: Loading: sentiment
2021-03-04 09:03:57 INFO: Loading: ner
2021-03-04 09:03:58 INFO: Done loading processors!


The preceding code instantiates an English pipeline and processes a sample piece of text. The next piece of code is a reusable function to print back the sentence tokens with the POS tags:

In [ ]:
def print_pos(doc):
    text = ""
    for sentence in doc.sentences:
        for token in sentence.tokens:
          text += token.words[0].text + "/" + token.words[0].upos + " "
        text += "\n"
    return text

In [ ]:
print(print_pos(pos))

Yo/PRON you/PRON around/ADV ?/PUNCT 
A/DET friend/NOUN of/ADP mine/PRON 's/PART lookin/NOUN ./PUNCT 



Most of these tags would make sense, though there may be some inaccuracies. For
example, the word lookin is miscategorized as a noun. Neither StanfordNLP, nor
a model from another package, will be perfect. This is something that we have to
account for in building models using such features. There are a couple of different features that can be built using these POS.

As a next step, let's update the word_counts() method and add a feature to show
the percentages of symbols and punctuation in a message – with the hypothesis that maybe spam messages use more punctuation and symbols. Other features around
types of different grammatical elements can also be built.

In [ ]:
en_sw = stopwords.stopwords('en')

def word_counts_v3(x, pipeline=en):
  doc = pipeline(x)
  totals = 0.
  count = 0.
  non_word = 0.
  for sentence in doc.sentences:
    totals += len(sentence.tokens)  # (1)
    for token in sentence.tokens:
        if token.text.lower() not in en_sw:
          if token.words[0].upos not in ['PUNCT', 'SYM']:
            count += 1.
          else:
            non_word += 1.
  non_word = non_word / totals
  return pd.Series([count, non_word], index=['Words_NoPunct', 'Punct'])

In [ ]:
x = train[:10]
x.describe()

,Spam,Capitals,Punctuation,Length
count,10.0,10.000000,10.000000,10.00000
mean,0.0,14.400000,18.300000,72.70000
std,0.0,32.948445,14.772723,50.36103
min,0.0,1.000000,4.000000,23.00000
25%,0.0,1.000000,7.250000,37.75000
50%,0.0,1.500000,13.000000,57.00000
75%,0.0,9.000000,23.750000,88.00000
max,0.0,107.000000,48.000000,161.00000


Since there are multiple computations that need to be performed on the message in each row, these operations are combined and a Series object with column labels is returned. This can be merged with the main DataFrame like so:

In [ ]:
train_tmp = train['Message'].apply(word_counts_v3)
train = pd.concat([train, train_tmp], axis=1)
train.describe()

,Spam,Capitals,Punctuation,Length,Words_NoPunct,Punct
count,4459.000000,4459.000000,4459.000000,4459.000000,4459.000000,4459.000000
mean,0.132765,5.519399,18.886522,80.316439,6.444046,0.148006
std,0.339359,11.405424,14.602023,59.346407,5.601280,0.095298
min,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,0.000000,1.000000,8.000000,35.000000,3.000000,0.090909
50%,0.000000,2.000000,15.000000,61.000000,5.000000,0.142857
75%,0.000000,4.000000,27.000000,122.000000,9.000000,0.200000
max,1.000000,129.000000,253.000000,910.000000,55.000000,0.818182


A similar process can be performed on the test set:

In [ ]:
test_tmp = test['Message'].apply(word_counts_v3)
test = pd.concat([test, test_tmp], axis=1)
test.describe()

,Spam,Capitals,Punctuation,Length,Words_NoPunct,Punct
count,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000
mean,0.139013,6.030493,19.166816,80.951570,6.611659,0.151337
std,0.346116,12.731059,15.694599,61.807655,5.828400,0.102153
min,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,0.000000,1.000000,8.000000,36.000000,3.000000,0.093750
50%,0.000000,2.000000,15.000000,61.000000,4.000000,0.142857
75%,0.000000,4.000000,28.000000,123.000000,9.000000,0.200000
max,1.000000,127.000000,195.000000,790.000000,46.000000,1.000000


In [ ]:
z = pd.concat([x, train_tmp], axis=1)
z.describe()

,Spam,Capitals,Punctuation,Length,Words_NoPunct,Punct
count,10.0,10.000000,10.000000,10.00000,4459.000000,4459.000000
mean,0.0,14.400000,18.300000,72.70000,6.444046,0.148006
std,0.0,32.948445,14.772723,50.36103,5.601280,0.095298
min,0.0,1.000000,4.000000,23.00000,0.000000,0.000000
25%,0.0,1.000000,7.250000,37.75000,3.000000,0.090909
50%,0.0,1.500000,13.000000,57.00000,5.000000,0.142857
75%,0.0,9.000000,23.750000,88.00000,9.000000,0.200000
max,0.0,107.000000,48.000000,161.00000,55.000000,0.818182


A quick check of the statistics for spam and non-spam messages in the training set shows the following, first for non-spam messages:

In [ ]:
z.loc[z['Spam']==0].describe()

,Spam,Capitals,Punctuation,Length,Words_NoPunct,Punct
count,10.0,10.000000,10.000000,10.00000,10.000000,10.000000
mean,0.0,14.400000,18.300000,72.70000,5.600000,0.152905
std,0.0,32.948445,14.772723,50.36103,8.058122,0.063198
min,0.0,1.000000,4.000000,23.00000,1.000000,0.000000
25%,0.0,1.000000,7.250000,37.75000,1.250000,0.132775
50%,0.0,1.500000,13.000000,57.00000,2.000000,0.177083
75%,0.0,9.000000,23.750000,88.00000,6.750000,0.196875
max,0.0,107.000000,48.000000,161.00000,27.000000,0.208333


In [ ]:
z.loc[z['Spam']==1].describe()

,Spam,Capitals,Punctuation,Length,Words_NoPunct,Punct
count,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
aa = [word_counts_v3(y) for y in x['Message']]

In [ ]:
ab = pd.DataFrame(aa)
ab.describe()

,Words_NoPunct,Punct
count,10.000000,10.000000
mean,5.600000,0.152905
std,8.058122,0.063198
min,1.000000,0.000000
25%,1.250000,0.132775
50%,2.000000,0.177083
75%,6.750000,0.196875
max,27.000000,0.208333


In general, word counts have been reduced even further after stop word removal.
Further more, the new Punct feature computes the ratio of punctuation tokens in a message relative to the total tokens. Now we can build a model with this data.

### Modeling data with POS tagging

Plugging these features into the model, the following results are obtained:

In [ ]:
x_train = train[["Length", "Punctuation", "Capitals", "Words_NoPunct", "Punct"]]
y_train = train[["Spam"]]

x_test = train[["Length", "Punctuation", "Capitals", "Words_NoPunct", "Punct"]]
y_test = train[["Spam"]]

# make build
model = make_model(input_dims=5)

# train the model
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 4s 3ms/step - loss: 9.6284 - accuracy: 0.3422
Epoch 2/10
446/446 [==============================] - 1s 3ms/step - loss: 0.5432 - accuracy: 0.8596
Epoch 3/10
446/446 [==============================] - 1s 3ms/step - loss: 0.4295 - accuracy: 0.8820
Epoch 4/10
446/446 [==============================] - 1s 3ms/step - loss: 0.3750 - accuracy: 0.8912
Epoch 5/10
446/446 [==============================] - 1s 3ms/step - loss: 0.3257 - accuracy: 0.9034
Epoch 6/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2850 - accuracy: 0.9185
Epoch 7/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2855 - accuracy: 0.9153
Epoch 8/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2460 - accuracy: 0.9266
Epoch 9/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2258 - accuracy: 0.9374
Epoch 10/10
446/446 [==============================] - 1s 3ms/step - loss: 0.2165 - accuracy: 0.9287

The accuracy shows a slight increase and is now up to 94.66%. Upon testing, it
seems to hold:

In [ ]:
model.evaluate(x_test, y_test)

140/140 [==============================] - 0s 2ms/step - loss: 0.2007 - accuracy: 0.9397


[0.20073966681957245, 0.939672589302063]